In [1]:
# optional step
# !pip install wandb
!wandb login c91e8fdad4598c56c9a7930b7039c3a887d10d9c
# !wandb status

# setup discord notification using discordwebhook
import discordwebhook

def send_notification(message):
    url = 'https://discord.com/api/webhooks/1100082805897187539/gshRvxkndkh9VIqr6C--wZip6LzhOhEUhvPzS6Jcddwp4Lj9KvzgZK4fHU8sO-NlmBIJ'
    discord = discordwebhook.Discord(url=url)
    discord.post(
        content=message + '\n @everyone',
        username='ABSA Notification'
    )

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\LENOVO/.netrc


In [2]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.0
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


# APC Model List

In [3]:
from pyabsa import AspectPolarityClassification as APC

[2023-04-27 18:38:35] (2.2.3) PyABSA(2.2.3): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)



C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


# APC Dataset List

In [5]:
# optional step
# from pyabsa import download_all_available_datasets
# print('Downloading all available datasets...')
# download_all_available_datasets()
# print('Download completed!')

In [6]:
dataset = APC.APCDatasetList.TripAdvisorAttractionId
dataset

['502.attraction_id']

# Training

In [7]:
from pyabsa import ModelSaveOption, DeviceTypeOption
!set CUDA_LAUNCH_BLOCKING=1
!set TORCH_USE_CUDA_DSA = 1

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [19]:
torch.backends.cuda.max_split_size_mb = 6000

In [26]:
from pyabsa.tasks.AspectPolarityClassification.models import APCModelList
config = APC.APCConfigManager.get_apc_config_indonesia()
config.model = APCModelList.FAST_LSA_T_V2
config.optimizer = 'adamw'
config.learning_rate = 0.00002
config.pretrained_bert = 'indobenchmark/indobert-large-p2'
config.use_bert_spc = True
config.max_seq_len = 80
config.dropout = 0.5
config.lsa = True
config.window = 'lr'
config.num_epoch = 10
config.batch_size = 8
config.initializer = 'xavier_uniform_'
config.seed = 52
config.output_dim = 3
config.log_step = 5

In [27]:
import wandb
wandb.init(project="pyabsa", entity="farrel-arrizal", config=config)

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\wandb\sdk\lib\ipython.py:59: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [28]:
send_notification('Training started!')
try:
    trainer = APC.APCTrainer(
    config=config,
    dataset=dataset,
    #from_checkpoint="english",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save=None,  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=False,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
    )
    send_notification('Training completed!')
    wandb.finish()
except Exception as e:
    send_notification(f'Training failed! {e}')
    wandb.finish()
    raise e

[2023-04-27 19:20:23] (2.2.3) Set Model Device: cuda:0
[2023-04-27 19:20:23] (2.2.3) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU
2023-04-27 19:20:24,576 INFO: PyABSA version: 2.2.3
2023-04-27 19:20:24,577 INFO: Transformers version: 4.28.1
2023-04-27 19:20:24,578 INFO: Torch version: 2.0.0+cuda11.8
2023-04-27 19:20:24,578 INFO: Device: NVIDIA GeForce RTX 3070 Laptop GPU
2023-04-27 19:20:24,579 INFO: AttractionReviewId in the trainer is not a exact path, will search dataset in current working directory
2023-04-27 19:20:25,638 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


2023-04-27 19:20:29,771 INFO: Load dataset from integrated_datasets\apc_datasets\500.Tripadvisor\502.attraction_id\train.tourism_review_id.txt.apc


preparing dataloader:   0%|          | 0/6931 [00:00<?, ?it/s]Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
preparing dataloader:  27%|██▋       | 1881/6931 [00:01<00:03, 1477.72it/s]

2023-04-27 19:20:31,080 WARNING: AspectEqualsTextWarning -> <aspect: ceking ricefield terrace tegallalang> equals <text:  ceking ricefield terrace tegallalang >, <polarity: Neutral>


...

preparing dataloader: 100%|██████████| 1729/1729 [00:01<00:00, 1551.99it/s]

2023-04-27 19:20:38,921 INFO: Dataset Label Details: {'Negative': 214, 'Neutral': 384, 'Positive': 1131, 'Sum': 1729}


2023-04-27 19:20:39,481 INFO: test data examples:
 [{'ex_id': tensor(0), 'text_raw': 'saya tidak yakin apa yang lebih menyenangkan di tempat ini - pemandangan atau kachak bertari... keduanya merupakan hal-hal yang anda harus melihat ketika di bali! tarian ini menjadi hanya 1 jam-bermain, sinopsis semua pararasul tersedia dalam bahasa inggris, dan harus jujur tanpa itu, itu adalah sangat sulit untuk bayangan tersebut.', 'text_spc': '[CLS] saya tidak yakin apa yang lebih menyenangkan di tempat ini - pemandangan atau kachak bertari... keduanya merupakan hal-hal yang anda harus melihat ketika di bali! tarian ini menjadi hanya 1 jam-bermain, sinopsis semua pararasul tersedia dalam bahasa inggris, dan harus jujur tanpa itu, itu adalah sangat sulit untuk bayangan tersebut. [SEP] tempat [SEP]', 'aspect': 'tempat', 'aspect_position': tensor(0, dtype=torch.int32), 'lca_ids': tensor([0.9104, 0.9254, 0.9403, 0.9552, 0.9701, 0.9851, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.

Epoch:0 | Loss:0:   0%|          | 0/867 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 8.00 GiB total capacity; 7.00 GiB already allocated; 0 bytes free; 7.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## load trained model for inference

In [9]:
from pyabsa.tasks.AspectPolarityClassification import SentimentClassifier

sentiment_classifier = trainer.load_trained_model()
assert isinstance(sentiment_classifier, SentimentClassifier)

[2023-04-19 16:11:22] (2.2.3) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93/
[2023-04-19 16:11:22] (2.2.3) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93/fast_lsa_t_v2.config
[2023-04-19 16:11:22] (2.2.3) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93/fast_lsa_t_v2.state_dict
[2023-04-19 16:11:22] (2.2.3) model: None
[2023-04-19 16:11:22] (2.2.3) tokenizer: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93/fast_lsa_t_v2.tokenizer
[2023-04-19 16:11:23] (2.2.3) Set Model Device: cuda:0
[2023-04-19 16:11:23] (2.2.3) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight', 'classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

[2023-04-19 16:11:28] (2.2.3) Caching dataset... please remove cached dataset if any problem happens.


In [18]:
from pyabsa import available_checkpoints
import warnings
warnings.filterwarnings('ignore')
ckpts = available_checkpoints()
# find a suitable checkpoint and use the name:
sentiment_classifier = APC.SentimentClassifier(
    checkpoint='checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93',  # you can also use the checkpoint path here
)
# load a trained model

[2023-04-19 16:22:51] (2.2.3) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2023-04-19 16:22:51] (2.2.3) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93
[2023-04-19 16:22:51] (2.2.3) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93\fast_lsa_t_v2.config
[2023-04-19 16:22:51] (2.2.3) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93\fast_lsa_t_v2.state_dict
[2023-04-19 16:22:51] (2.2.3) model: None
[2023-04-19 16:22:51] (2.2.3) tokenizer: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_87.64_f1_79.93\fast_lsa_t_v2.tokenizer
[2023-04-19 16:22:51] (2.2.3) Set Model Device: cuda:0
[2023-04-19 16:22:51] (2.2.3) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight', 'classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
sentiment_classifier.predict(
    text="A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , [B-ASP]food[E-ASP] was good but for not being very busy the [B-ASP]service[E-ASP] was terrible . We had to order the second round of drinks twice because they forgot we had ordered them . In the end we had a wonderful [B-ASP]time[E-ASP] and we ' d go back . 100 % recommended , just try one of the other restaurants .",
    return_raw_result=True,
    print_result=True,
    ignore_error=True,
    eval_batch_size=32,
)

[2023-04-19 16:29:39] (2.2.3) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 4, text: A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , [B-ASP]food[E-ASP] was good but for not being very busy the [B-ASP]service[E-ASP] was terrible . We had to order the second round of drinks twice because they forgot we had ordered them . In the end we had a wonderful [B-ASP]time[E-ASP] and we ' d go back . 100 % recommended , just try one of the other restaurants .
[2023-04-19 16:29:39] (2.2.3) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 4, text: A bit expensive but totally worth every second . Riding the [B-ASP]elephant[E-ASP] was such a great time , 30 minutes of excitement , I felt like a

{'text': 'A bit expensive but totally worth every second . Riding the elephant was such a great time , 30 minutes of excitement , I felt like a little kid picking out their first pet ( it was my first time doing this . ) We then had lunch at the elephant view restaurant , food was good but for not being very busy the service was terrible . We had to order the second round',
 'aspect': ['elephant', 'food'],
 'sentiment': ['Positive', 'Positive'],
 'confidence': [0.9990125894546509, 0.9975094795227051],
 'probs': [array([8.2562224e-04, 1.6176124e-04, 9.9901259e-01], dtype=float32),
  array([2.1630367e-03, 3.2749053e-04, 9.9750948e-01], dtype=float32)],
 'ref_sentiment': ['-100', '-100'],
 'ref_check': ['', ''],
 'perplexity': 'N.A.'}

In [13]:
print('Training completed!')

Training completed!


In [25]:
sentiment_classifier.batch_predict(
    target_file=APC.APCDatasetList.TripAdvisorAttractionEn,  # the batch_predict() is only available for a file only, please put the examples in a file
    print_result=True,
    save_result=False,
    ignore_error=True,
    eval_batch_size=32,
)

[2023-04-19 16:32:25] (2.2.3) Try to load 501.attraction_en dataset from local disk


run inference: 100%|██████████| 12/12 [00:01<00:00,  6.91it/s]


[2023-04-19 16:32:29] (2.2.3) Example 0: An awesome and must see <place:Positive(confidence:1.0, ref:Positive)> when in Bali . <Views:Positive(confidence:1.0, ref:Positive)> are stunning . Be sure to plan to hang out at least 2 hours in this <place:Positive(confidence:1.0, ref:Positive)> .
[2023-04-19 16:32:29] (2.2.3) Example 1: There is a temple on a rock . But there is also an entrance <fee:Negative(confidence:0.808, ref:Negative)> of IDR 60 . 000 . There are also over 3000 other visitors . Not only do they pay this <fee:Negative(confidence:0.808, ref:Negative)> ( you do the math of the money they make out of this every day ! ) but also are they all in your way when you want to make pictures . Try and photoshop that . . .
[2023-04-19 16:32:29] (2.2.3) Example 2: We had a great day at <waterbom:Positive(confidence:0.998, ref:Positive)> Bali , splashed out on a gazebo which was useful , especially when it started to rain as it kept all our stuff dry whilst we enjoyed the lazy <river:P

[{'text': 'An awesome and must see place when in Bali . Views are stunning . Be sure to plan to hang out at least 2 hours in this place .',
  'aspect': ['place', 'Views'],
  'sentiment': ['Positive', 'Positive'],
  'confidence': [0.9997370839118958, 0.9997602105140686],
  'probs': [array([1.3971316e-04, 1.2327821e-04, 9.9973708e-01], dtype=float32),
   array([1.3144243e-04, 1.0831768e-04, 9.9976021e-01], dtype=float32)],
  'ref_sentiment': ['Positive', 'Positive'],
  'ref_check': ['Correct', 'Correct'],
  'perplexity': 'N.A.'},
 {'text': 'There is a temple on a rock . But there is also an entrance fee of IDR 60 . 000 . There are also over 3000 other visitors . Not only do they pay this fee ( you do the math of the money they make out of this every day ! ) but also are they all in your way when you want to make pictures . Try and photoshop that . . .',
  'aspect': ['fee'],
  'sentiment': ['Negative'],
  'confidence': [0.8078023195266724],
  'probs': [array([8.0780232e-01, 1.9185939e-01,